In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime
from numpy import pi, sin
from operator import add
from time import sleep

from matplotlib import animation
from matplotlib.widgets import Slider, Button, RadioButtons

#from sympy import *
from sympy import exp, init_printing, integrate, latex, Lambda, N, var

#from IPython import display
from IPython.display import clear_output, display, HTML, Latex

init_printing(use_latex=True)

%matplotlib inline
# %pylab inline

In [ ]:
plt.rcParams["font.size"] = 30
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.titlesize'] = 20
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 20
plt.rcParams["ytick.labelsize"] = 20
plt.rcParams["legend.fontsize"] = 20

In [ ]:
BASE_DIR = "."
DEBUG = False

rand_state = np.random.RandomState(42)

# Probability Density Function

In [ ]:
init_printing(use_latex=True)

fig = plt.gcf()
fig.set_size_inches(10,8)
var('a b x sigma mu')
pdf = Lambda((x,mu,sigma),
  (1/(sigma * np.sqrt(2*np.pi)) * exp(-(mu-x)**2 / (2*sigma**2)))
)
cdf = Lambda((a,b,mu,sigma),
  integrate(
    pdf(x,mu,sigma),(x,a,b)
  )
)

ltx = '$ pdf(x,\\mu,\\sigma) = \\frac{1}{ \\sigma' + \
 '\\sqrt{2 \\pi}} e^{\\left(-\\frac{{\\left(\\mu - ' + \
 'x\\right)}^{2}}{2 \\, \\sigma^{2}}\\right)}$'
display(Latex(ltx))

display(Latex('$ cdf(a,b,\mu,\sigma) = ' + latex(cdf(a,b,mu,sigma)) + '$'))


mu = 50
sigma = 15
x = np.linspace(0,100,100)
y = np.array([pdf(v,mu,sigma) for v in x],dtype='float')

plt.grid(True)
plt.title('Probability Density Function',fontsize=30)
plt.xlabel('Score',fontsize=20)
plt.ylabel('Density',fontsize=20)
plt.text(7,0.023,'$\mu = 50$',fontsize=15)
plt.text(7,0.021,'$\sigma = 15$',fontsize=15)
plt.text(50-2,-0.0028,'50',fontsize=20,color="green")
plt.tick_params(labelsize=20)

plt.plot(x,y,color='gray')
plt.fill_between(x,y,0,color='#c0f0c0')

plt.axvline(mu, linestyle=':', linewidth=3, label='$\mu$', color="green")

ymax=max(y)*2/3

plt.vlines(x=mu+sigma, ymin=0, ymax=ymax, linestyle='-', linewidth=2, label='$\mu \pm \sigma$', color="blue")
plt.vlines(x=mu-sigma, ymin=0, ymax=ymax, linestyle='-', linewidth=2, color="blue")
           
plt.vlines(x=mu+2*sigma, ymin=0, ymax=ymax, linestyle='-', linewidth=2, label='$\mu \pm 2\sigma$', color="brown")
plt.vlines(x=mu-2*sigma, ymin=0, ymax=ymax, linestyle='-', linewidth=2, color="brown")

plt.vlines(x=mu+3*sigma, ymin=0, ymax=ymax, linestyle='-', linewidth=2, label='$\mu \pm 3\sigma$', color="red")
plt.vlines(x=mu-3*sigma, ymin=0, ymax=ymax, linestyle='-', linewidth=2, color="red")

plt.legend(prop=dict(size=15))

plt.show()

tbl = [['$\pm \sigma$','min score','max score','% of scores in range']]
for s in range(0,4):
  tbl.append(['%d' % s, '%d' % (mu-s*sigma), '%d' % (mu+s*sigma),'%.4f' % N(cdf(-s,s,0,1))])
df = pd.DataFrame(tbl)
HTML(df.to_html(header=None,index=None, justify="center"))

# Cumulative Distribution Function

In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,5)
display(Latex('$ cdf(a,b,\mu,\sigma) = ' + latex(cdf(a,b,mu,sigma)) + '$'))
mu = 50
sigma = 15
x = np.linspace(0,100,100)
y = np.array([cdf(-1e99,v,mu,sigma) for v in x],dtype='float')
plt.grid(True)
plt.title('Cumulative Distribution Function',fontsize=30)
plt.xlabel('Score',fontsize=20)
plt.ylabel('Cumulative Density',fontsize=20)
plt.text(3,.73,'$\mu = 50$',fontsize=16)
plt.text(3,.63,'$\sigma = 15$',fontsize=16)
plt.plot(x,y,color='gray')
plt.fill_between(x,y,0,color='#c0f0c0')
plt.show()

If you want to use symbols instead of numerical values in formula, you need to create the Latex expression in the display before you assign the values to $\mu$ and $\sigma$

In [ ]:
fig = plt.gcf()
fig.set_size_inches(8,5)
var('a b x sigma mu')
pdf = Lambda((x,mu,sigma),
  (1/(sigma * np.sqrt(2*pi)) * exp(-(mu-x)**2 / (2*sigma**2)))
)
cdf = Lambda((a,b,mu,sigma),
  integrate(
    pdf(x,mu,sigma),(x,a,b)
  )
)

display(Latex('$ cdf(a,b,\mu,\sigma) = ' + latex(cdf(a,b,mu,sigma)) + '$'))

# To display symbols, the above line must be before the following assignments

mu = 50
sigma = 15

x = np.linspace(0,100,100)
y = np.array([cdf(-1e99,v,mu,sigma) for v in x],dtype='float')
plt.grid(True)
plt.title('Cumulative Distribution Function',fontsize=30)
plt.xlabel('Score',fontsize=20)
plt.ylabel('Cumulative Density',fontsize=20)
plt.text(3,.73,'$\mu = 50$',fontsize=16)
plt.text(3,.63,'$\sigma = 15$',fontsize=16)
plt.plot(x,y,color='gray')
plt.fill_between(x,y,0,color='#c0f0c0')
plt.show()

# Random Walk & Normal Distribution

In [ ]:
def random_walk_1D(T, N):
    dt = T/float(N)
    # simulate the increments by normal random variable generator
    increments = rand_state.normal(0, 1*np.sqrt(dt), N)
    t = np.linspace(0.0, N*dt, N+1)
    W = [0] + list(np.cumsum(increments))
    return t, W

In [ ]:
T = 10     # T: total time
N = 500    # N: The total number of steps

In [ ]:
plt.xlabel('time')
plt.ylabel('walk')
#plt.grid(True)

for i in range(100):
    t, W = random_walk_1D(T, N)

    # scatter points on the main axes
    plt.plot(t, W, markersize=1, alpha=1)

    display(plt.gcf())
    clear_output(wait=True)
#    sleep(1)

Let's repeat the above, but this time also plotting the histogram:

In [ ]:
start = datetime.now()
print("Start time : {0}".format(start.strftime("%Y-%m-%d %H:%M:%S")))

# reset the RNG for repeatability
rand_state = np.random.RandomState(0)

T = 10     # T: total time
N = 500    # N: The total number of steps

fig = plt.figure(figsize=(15, 10))
grid = plt.GridSpec(1, 7, hspace=0.01, wspace=0.01)
main_ax = fig.add_subplot(grid[0, 0:5], xticklabels=[0, 2, 4, 6, 8, ], yticklabels=[-15, -10, -5, 0, 5, 10, 15])
y_hist = fig.add_subplot(grid[0, 5:7], xticklabels=[0, 100, 200, 300], yticklabels=[]) #, sharey=main_ax)
#y_hist.invert_xaxis()

main_ax.set_xlabel('time')
main_ax.set_ylabel('walk')

#main_ax.yaxis.tick_right()

main_ax.set_xlim((0, 10))
main_ax.set_ylim((-15, 15))

y_hist.set_xlim((0, 300))
y_hist.set_ylim((-15, 15))

y_hist.set_xlabel('count')
#y_hist.set_ylabel('walk')

#y_hist.yaxis.tick_right()

#plt.grid(True)
y = []

while True:
    try:
        t, W = random_walk_1D(T, N)
        y.append(W[-1])

        # scatter points on the main axes
        main_ax.plot(t, W, markersize=1, alpha=1)
        
        # histogram on the attached axes
        y_hist.hist(y, 40, histtype='stepfilled', orientation='horizontal', color='red')
        
        display(plt.gcf())
        clear_output(wait=True)
        #sleep(1)
    except KeyboardInterrupt:
        break
        
end = datetime.now()
print("Start time : {0}".format(start.strftime("%Y-%m-%d %H:%M:%S")))
print("End time : {0}   (total run-time={1:.2f}sec)".format(end.strftime("%Y-%m-%d %H:%M:%S"),
                                              (datetime.now()-start).total_seconds()))

Here is how my output looks like after 1 hour:

<IMG src="Random_Walk_after_1_hour.png">

Here is how my output looks like after 3 hours:

<IMG src="Random_Walk_after_3_hours.png">

Here is how my output looks like after 7 hours:

<IMG src="Random_Walk_after_7_hours.png">